In [1]:
import BoWTfidfVectorizer
import preprocess_new
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import numpy as np
from sklearn import preprocessing
from glove import Corpus, Glove

#For BoW with Tf-Idf
#preProcessedTrDF= preprocess_new.prepareTrainTestSet('train.csv','test.csv','bow',seperateLabelInfo=1,tokenize=0)
#embed_vect = BoWTfidfVectorizer.getBoWTfidfVectors(preProcessedTrDF['comment_text'].values)

#For using Glove Embedding and Paragraf Vectors
glove = Glove.load('glove.model')
preProcessedTrDF= preprocess_new.prepareTrainTestSet('train.csv','test.csv','glove',seperateLabelInfo=1,tokenize=1)
embed_vect = list(map(lambda x:glove.transform_paragraph(x,ignore_missing=True),preProcessedTrDF['comment_text'].values))


100%|██████████| 159548/159548 [36:38<00:00, 72.56it/s]
/homes/fo301/.local/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/homes/fo301/.local/lib/python3.4/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [15]:
#Preparing Train and Test Dataset, splitting according to 70% train set and 30% test set
features={"toxic","severe_toxic","obscene","threat","insult","identity_hate"}
X_tr={}
X_te={}
y_tr={}
y_te={}
preProcessedTrDF=preProcessedTrDF.reset_index()
for feature in features:
    X_tr[feature], X_te[feature], y_tr[feature], y_te[feature] = train_test_split(embed_vect, preProcessedTrDF[feature].values.astype(np.float), test_size = .3, random_state=25)
y_tr['identity_hate'][41272]=0    

In [16]:
#Further Cleaning the Dataset for Glove vectorizer, by leaving out NaN values. 
#The Sklearn CountVectorizer and Tf-Idf Vectorizers deals with problems itself, but by Glove vectors the vectorizer produces NaN and negative values
for feature in features:
    ary=[]
    a=-1
    for i in X_tr[feature]:
        a=a+1
        ary.append(not np.isnan(i).any())
        if (np.isnan(i).any()):
            print(a)
    ary=np.array(ary)
    X_tr[feature]=np.array(X_tr[feature])[ary]
    y_tr[feature]=y_tr[feature][ary]

for feature in features:
    ary2=[]
    a=-1
    for i in X_te[feature]:
        a=a+1
        ary2.append(not np.isnan(i).any())
        if (np.isnan(i).any()):
            print(a)
    ary2=np.array(ary2)
    X_te[feature]=np.array(X_te[feature])[ary2]
    y_te[feature]=y_te[feature][ary2]
#y_tr[feature][41272]
#Scaling the values between the values 0 and 1.
for feature in features:
    min_max_scaler = preprocessing.MinMaxScaler()
    X_tr[feature] = min_max_scaler.fit_transform(X_tr[feature])
    X_te[feature] = min_max_scaler.transform(X_te[feature])


587
2132
2690
5036
5491
6456
10168
11994
17998
18634
18909
19462
20823
23291
26010
26777
27674
35816
38310
41272
42264
45398
46467
48091
48386
48828
49248
52658
55599
57262
61696
65280
67843
68105
68180
69024
69617
72540
74018
77333
78699
79945
83428
84924
86036
87780
88069
88753
93316
97550
99150
99910
100047
101128
101921
102340
105246
105723
107476
111078
587
2132
2690
5036
5491
6456
10168
11994
17998
18634
18909
19462
20823
23291
26010
26777
27674
35816
38310
41272
42264
45398
46467
48091
48386
48828
49248
52658
55599
57262
61696
65280
67843
68105
68180
69024
69617
72540
74018
77333
78699
79945
83428
84924
86036
87780
88069
88753
93316
97550
99150
99910
100047
101128
101921
102340
105246
105723
107476
111078
587
2132
2690
5036
5491
6456
10168
11994
17998
18634
18909
19462
20823
23291
26010
26777
27674
35816
38310
41272
42264
45398
46467
48091
48386
48828
49248
52658
55599
57262
61696
65280
67843
68105
68180
69024
69617
72540
74018
77333
78699
79945
83428
84924
86036
87780
88069
887

In [17]:
#Defining the Classifier as MultinomialNB and training the train-set using fit function.
pipeline={}
for feature in features:
    pipeline[feature] =  Pipeline([('mNB', MultinomialNB())])
    pipeline[feature].fit(X_tr[feature],y_tr[feature])

In [18]:
#Prediction using test-set and report of statistics.
predicted={}
accF={}
for feature in features:
    predicted[feature] = pipeline[feature].predict(X_te[feature])
    print('\n---------------------------------------------------------------------------------')
    print('Statistics for : '+feature+'\n')
    
    print(metrics.classification_report(y_te[feature], predicted[feature]))
    
    confusion_matrixF = confusion_matrix(y_te[feature], predicted[feature])
    print("Confusion Matrix\n:")
    print(confusion_matrixF)
    
    print("\nNumber of samples in the test set: "+str(len(y_te[feature])))
    print("Number of "+feature+" comments (target = 1) in the test set: "+str(sum(y_te[feature]))+" ("+str(round(sum(y_te[feature])/len(y_te[feature])*100,2))+"% of the total)")

    print(classification_report(y_te[feature], predicted[feature]))

    accF[feature] = accuracy_score(y_te[feature], predicted[feature], normalize=True, sample_weight=None)
    print("Accuracy: "+str(round(accF[feature],2)*100)+'%')
    
    print('\n---------------------------------------------------------------------------------')


---------------------------------------------------------------------------------
Statistics for : threat

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     47682
        1.0       0.00      0.00      0.00       154

avg / total       0.99      1.00      1.00     47836

Confusion Matrix
:
[[47682     0]
 [  154     0]]

Number of samples in the test set: 47836
Number of threat comments (target = 1) in the test set: 154.0 (0.32% of the total)
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     47682
        1.0       0.00      0.00      0.00       154

avg / total       0.99      1.00      1.00     47836

Accuracy: 100.0%

---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Statistics for : identity_hate

             precision    recall  f1-score   support

        0.0       

/homes/fo301/.local/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 99.0%

---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Statistics for : toxic

             precision    recall  f1-score   support

        0.0       0.90      1.00      0.95     43150
        1.0       0.00      0.00      0.00      4686

avg / total       0.81      0.90      0.86     47836

Confusion Matrix
:
[[43150     0]
 [ 4686     0]]

Number of samples in the test set: 47836
Number of toxic comments (target = 1) in the test set: 4686.0 (9.8% of the total)
             precision    recall  f1-score   support

        0.0       0.90      1.00      0.95     43150
        1.0       0.00      0.00      0.00      4686

avg / total       0.81      0.90      0.86     47836

Accuracy: 90.0%

---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Statisti

In [ ]:
#embed_vect = list(map(lambda x:glove.transform_paragraph(x,ignore_missing=True),preProcessedTrDF['comment_text'].astype(np.float).values))